In [1]:
from openai import OpenAI

openai_api_key = "sk-KZruNT10C9eESLJW24467b8cF5Ef463184A010AeAcB58a74"
openai_api_base = "http://47.88.65.188:8300/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
input_price_per_token = 0.000150 / 1000  
output_price_per_token = 0.000600 / 1000 
total_cost = 0


In [2]:
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
# 读取 JSONL 文件并转换为字典列表
def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {line_number}: {e}")
                print(f"Line content: {line}")
    return data
def write_jsonl_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as file:
        for entry in data:
            file.write(json.dumps(entry) + '\n')


In [3]:
def get_answer(problem):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": "You will receive an OOP code generation problem. Please use Python to generate the code that meets the requirements and return it. Only return the code, and do not include any other content. Omit '```python' at the beginning and '```' at the end.\n"
                +f"Problem: {problem['question']}"
            }
        ]
    )
    input_tokens = completion.usage.prompt_tokens
    output_tokens = completion.usage.completion_tokens
    global total_cost
    cost = (input_tokens * input_price_per_token) + (output_tokens * output_price_per_token)
    total_cost += cost 
    return completion.choices[0].message.content


def ask(problem, itr = 1):
    results = []
    for i in range(itr):
        print(f"Problem_id:{problem['task_id']}-itr:{i}")
        results.append(get_answer(problem))

    return problem['task_id'], results
    


In [4]:
file_path = '../data/oop_data_difficulty.jsonl'
problems = read_jsonl_file(file_path)

In [5]:
new_result = []
with ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(ask, problem) for problem in problems]
    for future in as_completed(futures):
        try:
            task_id, results = future.result()
            for i in range(len(results)):
                new_entry = {
                    "task_id": task_id,
                    "completion": results[i]
                }
                new_result.append(new_entry)
        except Exception as e:
            print(f"An error occurred: {e}")
            

new_result.sort(key=lambda x: int(x['task_id'].split('/')[1]))
out_file_path = 'result_difficulty.jsonl'
write_jsonl_file(out_file_path, new_result)
print(f"Total cost: {total_cost} USD")

Problem_id:OOP/0-itr:0
Problem_id:OOP/1-itr:0
Problem_id:OOP/2-itr:0
Problem_id:OOP/3-itr:0
Problem_id:OOP/4-itr:0
Problem_id:OOP/5-itr:0
Problem_id:OOP/6-itr:0
Problem_id:OOP/7-itr:0
Problem_id:OOP/8-itr:0
Problem_id:OOP/9-itr:0
Problem_id:OOP/10-itr:0
Problem_id:OOP/11-itr:0
Problem_id:OOP/12-itr:0
Problem_id:OOP/13-itr:0
Problem_id:OOP/14-itr:0
Problem_id:OOP/15-itr:0
Problem_id:OOP/16-itr:0
Problem_id:OOP/17-itr:0
Problem_id:OOP/18-itr:0
Problem_id:OOP/19-itr:0
Problem_id:OOP/20-itr:0
Problem_id:OOP/21-itr:0
Problem_id:OOP/22-itr:0
Problem_id:OOP/23-itr:0
Problem_id:OOP/24-itr:0
Problem_id:OOP/25-itr:0
Problem_id:OOP/26-itr:0
Problem_id:OOP/27-itr:0
Problem_id:OOP/28-itr:0
Problem_id:OOP/29-itr:0
Problem_id:OOP/30-itr:0
Problem_id:OOP/31-itr:0
Problem_id:OOP/32-itr:0
Problem_id:OOP/33-itr:0
Problem_id:OOP/34-itr:0
Problem_id:OOP/35-itr:0
Problem_id:OOP/36-itr:0
Problem_id:OOP/37-itr:0
Problem_id:OOP/38-itr:0
Problem_id:OOP/39-itr:0
Problem_id:OOP/40-itr:0
Problem_id:OOP/41-itr:0
Pr

In [6]:
difficuilt_result_file = 'result_difficulty.jsonl_results_oop_rule_matching.jsonl'
difficuilt_result = read_jsonl_file(difficuilt_result_file)
for i in range(len(difficuilt_result)):
    print(f'{difficuilt_result[i]["task_id"]}-{difficuilt_result[i]["result"]}')

OOP/0-passed
OOP/1-passed
OOP/2-failed: 
OOP/3-passed
OOP/4-failed: 
OOP/5-failed: 
OOP/6-failed: 
OOP/7-passed
OOP/8-passed
OOP/9-failed: 
OOP/10-failed: 
OOP/11-passed
OOP/12-failed: 
OOP/13-passed
OOP/14-failed: 
OOP/15-failed: 
OOP/16-failed: 
OOP/17-passed
OOP/18-passed
OOP/19-passed
OOP/20-failed: 
OOP/21-failed: 
OOP/22-failed: 
OOP/23-failed: 
OOP/24-failed: 
OOP/25-failed: 
OOP/26-passed
OOP/27-timed out
OOP/28-failed: 
OOP/29-failed: 
OOP/30-failed: 
OOP/31-failed: 
OOP/32-failed: 
OOP/33-failed: 
OOP/34-failed: 
OOP/35-passed
OOP/36-failed: 
OOP/37-failed: 
OOP/38-failed: 
OOP/39-failed: 
OOP/40-passed
OOP/41-passed
OOP/42-passed
OOP/43-failed: 
OOP/44-passed
OOP/45-failed: 
OOP/46-failed: 
OOP/47-passed
OOP/48-passed
OOP/49-failed: 
OOP/50-failed: 
OOP/51-failed: 
OOP/52-failed: 
OOP/53-failed: 
OOP/54-failed: 
OOP/55-failed: 
OOP/56-failed: 
OOP/57-failed: 
OOP/58-failed: 
OOP/59-failed: 
OOP/60-passed
OOP/61-failed: 
OOP/62-passed
OOP/63-failed: 
OOP/64-failed: 
OOP/65-pa